In [2]:
import pandas as pd
import numpy as np
import os

ImportError: lxml not found, please install or use the etree parser.

In [21]:
# read the taxon and vernacular names
# replace the id in the description with the correct names


taxon_columns = ['id','references','modified','scientificName','scientificNameAuthorship','taxonRank','verbatimTaxonRank',
                 'kingdom','phylum','class','order','family','genus','subgenus','taxonRemarks','trend','fossilRange',
                 'taxobox','acceptedNameUsage','acceptedNameUsageID','taxonomicStatus']
taxon_data = pd.read_table(filepath_or_buffer= "/mnt/c/Users/mikeha/Work/Biodiversity/AI/GBIF_wikipedia_records/wikipedia-en-dwca/taxon.txt", sep = '\t', header = None,names=taxon_columns)


/tmp/ipykernel_23609/2455920880.py:8: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  taxon_data = pd.read_table(filepath_or_buffer= "/mnt/c/Users/mikeha/Work/Biodiversity/AI/GBIF_wikipedia_records/wikipedia-en-dwca/taxon.txt", sep = '\t', header = None,names=taxon_columns)


In [22]:

vernacular_columns = ['id','isPreferredName','language','vernacularName']
vernacular_data = pd.read_table(filepath_or_buffer= "/mnt/c/Users/mikeha/Work/Biodiversity/AI/GBIF_wikipedia_records/wikipedia-en-dwca/vernacularname.txt", sep = '\t', header = None,names=vernacular_columns)
vernacular_data['id'] = vernacular_data['id'].astype(str)

In [23]:
#Select only species-level records

species_rows = np.where(taxon_data['taxonRank'] == 'Species')[0]
species_ids = taxon_data['id'][species_rows]
print(species_ids)

1              680
2              681
43            1043
44            1064
88            1367
            ...   
564249    71435279
564250    71435370
564251    71435719
564252    71435851
564253    71435888
Name: id, Length: 340317, dtype: object


In [24]:

taxon_data = taxon_data.iloc[species_rows,:]
#vernacular_data = vernacular_data.iloc[np.where(species_ids.isin(vernacular_data['id']))[0],:]


IndexError: positional indexers are out-of-bounds

In [25]:
#Join scientific name and vernacular names into one table by and with id
joint_vernacular_taxon = pd.merge(vernacular_data[['id','vernacularName']],taxon_data[['id','scientificName']], on='id')

In [26]:
joint_vernacular_taxon

,id,vernacularName,scientificName
0,680,Aardvark,Orycteropus afer
1,681,Aardwolf,Proteles cristata
2,1043,Northern cavefish,Amblyopsis spelaea
3,1543,African lily,Agapanthus africanus
4,2208,Arctic fox,Vulpes lagopus
...,...,...,...
32159,71378910,Caucasochasma,Caucasochasma zherikhini
32160,71392599,Sulaiman markhor,Capra falconeri jerdoni
32161,71394902,Bitter Vine,Trimenia moorei
32162,71401438,Paracapsulapagurus,Paracapsulapagurus poponguinensis


In [27]:
#read the description data and insert the scientific name and vernacular name
description_columns = ['id','language','type','description','references','license']
description_data = pd.read_table(filepath_or_buffer= "/mnt/c/Users/mikeha/Work/Biodiversity/AI/GBIF_wikipedia_records/wikipedia-en-dwca/description.txt", sep = '\t', header = None,names=description_columns)
description_data['id'] = description_data['id'].astype(str)

In [28]:
joint_description = pd.merge(joint_vernacular_taxon,description_data[['id','type','description']],on = 'id')

In [29]:
sentence_description = joint_description.loc[:,joint_description.columns != 'id'].apply(lambda x: '. '.join(x.dropna().values.tolist()), axis=1)

In [30]:
#write the sentence description to file for the retrieval dossier
sentence_description.to_csv('../data/retrieval_dossier/wikipedia-en-dwca-species-descriptions.csv',header = None)